In [14]:
import matplotlib.pyplot as plt

#%matplotlib notebook

#%matplotlib inline
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
mpl.rcParams['legend.fontsize'] = 10

import numpy as np

from numba import jit

In [22]:
@jit(nopython=True)
def normalize(v):
    return v / np.linalg.norm(v)

def cos_sim(a,b):
    return np.dot(a, b) / ( np.linalg.norm(a) * np.linalg.norm(b) )

In [76]:
cos_sim([2,3,4], [1, 0, 0])

0.3713906763541037

In [77]:
 2 / np.linalg.norm([2,3,4])

0.3713906763541037

In [3]:
smpl = np.random.randint(low=-1e+2, high=1e+2, size=(10, 3)) / 1e+2
smpls

array([[ 0.5 ,  0.75, -0.68],
       [-0.5 ,  0.02, -0.56],
       [-0.48,  0.08,  0.75],
       [ 0.68,  0.47, -0.8 ],
       [ 0.33, -0.44,  0.97],
       [-0.86, -0.82,  0.07],
       [ 0.21, -0.24, -0.29],
       [-0.2 , -0.48, -0.9 ],
       [ 0.24, -0.92, -0.37],
       [ 0.27, -0.11,  0.36]])

In [72]:
a = np.random.randint(10, size=10)
print(a, np.argsort(a), a[(a > 5) & (a < 9)])

[4 2 0 6 8 4 5 5 6 0] [2 9 1 0 5 6 7 3 8 4] [6 8 6]


In [73]:
@jit(nopython=True)
def two_point_standardization(struct, idx0, idx1):
    
    coo0 = struct[idx0]
    coo1 = struct[idx1]
    
    normal = (coo1 - coo0) / 2
    
    normal_unit = normalize(normal)
    
    vec0 = coo0 + normal
    
    # Plane Equation
    d = -vec0 @ normal
    plane = lambda x, y: (-normal[0] * x - normal[1] * y - d) / normal[2]
    #
    
    # make orthonormal basis on plane
    u_x = 1
    u_y = 0
    
    u = np.array([u_x, u_y, plane(u_x + vec0[0], u_y + vec0[1]) - vec0[2]])
    
    ## Cross product
    a1, a2, a3 = u
    b1, b2, b3 = normal
    
    v = np.array(
        [
            a2 * b3 - a3 * b2,
            a3 * b1 - a1 * b3,
            a1 * b2 - a2 * b1
        ]
    ) 
    ##
    
    u_unit = normalize(u)
    v_unit = normalize(v)
    #

    # Change of basis
    basis_matrix = np.vstack((normal_unit, u_unit, v_unit)).T    
    transform_matrix = np.linalg.inv(basis_matrix)
    #
    
    return transform_matrix @ (struct - vec0).T

In [74]:
np.around(two_point_standardization(smpl, 3,6).T, decimals=14)

array([[-0.54966034, -0.05104236, -0.34603929],
       [ 0.50774691, -0.70507663, -0.38384244],
       [ 1.12848879,  0.19739262, -1.10518062],
       [-0.49626102,  0.        , -0.        ],
       [ 1.2299475 ,  0.9421208 , -0.46487907],
       [ 1.60283594, -0.54286575, -0.30280145],
       [ 0.49626102,  0.        ,  0.        ],
       [ 0.5486528 , -0.71488138,  0.28982705],
       [ 0.92738495, -0.0321538 ,  0.53178783],
       [ 0.70885076,  0.48461402, -0.40367667]])

In [13]:
np.around(two_point_standardization(smpl, 3,6).T, decimals=14).T

array([[ 0.54966034, -0.50774691, -1.12848879,  0.49626102, -1.2299475 ,
        -1.60283594, -0.49626102, -0.5486528 , -0.92738495, -0.70885076],
       [-0.05104236, -0.70507663,  0.19739262, -0.        ,  0.9421208 ,
        -0.54286575, -0.        , -0.71488138, -0.0321538 ,  0.48461402],
       [ 0.34603929,  0.38384244,  1.10518062,  0.        ,  0.46487907,
         0.30280145, -0.        , -0.28982705, -0.53178783,  0.40367667]])

In [12]:
np.linalg.norm(np.around(two_point_standardization(smpl, 3,6), decimals=14), axis=0)

array([0.65151746, 0.94988157, 1.591815  , 0.49626102, 1.61755216,
       1.7191495 , 0.49626102, 0.94661238, 1.06952092, 0.94882822])

In [ ]:
def make4class(struct, idx0, idx1):
    
    standardized = np.around(two_point_standardization(struct, idx0, idx1), decimals=14)
    norm = np.linalg.norm(standardized, axis=0)
    
    #struct['relative_x'] = struct.T[0]
    #struct['relative_norm'] = norm
    
    
    
    
    
    

In [ ]:
def generator(idxes, batch_size=32):

    while True:

        x = np.empty((batch_size, , , ))
        y = np.empty((batch_size,))

        for i, row in enumerate(rows):
            
            for j in range(4):
                
                row += np.random.randint(-50, +50)
                segment = flatiron(train.acoustic_data.values[row-147456:row])
                smpl = np.empty((4096,36))
                
                for k in range(4096):
                    
                    smpl[k] = segment[k::4096]
                    
                x[(batch_size//4)*j+i] = smpl.reshape(288, 512)[:, 256:, np.newaxis]
                y[(batch_size//4)*j+i] = train.time_to_failure.values[row]

        yield x, y